# **Types of Sequential Data**


In common machine learning tasks, it is assumed that the data is identically and independently distributed (i.i.d.). However, when dealing with changes in distribution of the underlying data generating process, or working with data that has temporal dependence, this i.i.d. assumption breaks.

Practitioners and data scientists should be able to model such data by drawing from a variety of tools for sequential data analysis. In this lab, we introduce forms of sequential data, and basic concepts needed to understand components of a time-series.


## **Table of Contents**

<ol>
    <li><a href="https://#Objectives">Objectives</a></li>
    <li><a href="https://#Datasets">Datasets</a></li>
    <li>
        <a href="https://#Setup">Setup</a>
        <ol>
            <li><a href="https://#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="https://#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="https://#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li><a href="https://#Background">Background</a>
        <ol></ol>
    </li>
    <li><a href="https://#The i.i.d. Assumption">The i.i.d. Assumption</a>
        <ol></ol>
    </li>
    <li><a href="https://#Understanding Sequential Modeling">Understanding Sequential Modeling</a>
        <ol>
        </ol>
    </li>
    <li><a href="https://#Forms of Sequential Data">Forms of Sequential Data</a>
        <ol>
            <li><a href="https://#Working with time-series data">Working with time-series data</a></li>
            <li><a href="https://#Working with text data">Working with text data</a></li>
        </ol>
    </li>
    <li><a href="https://#Example 1 - Working with Speech Commands">Example 1 - Working with Speech Commands</a>
        <ol>
            <li><a href="https://#Exercise A - Import data set">Exercise A - Import data set</a></li>
            <li><a href="https://#Exercise B - Extract audio clips">Exercise B - Extract audio clips</a></li>
            <li><a href="https://#Exercise C - Plotting audio clips">Exercise C - Plotting audio clips</a></li>
            <li><a href="https://#Exercise D - Basic pre-processing">Exercise D - Basic pre-processing</a></li>
            <li><a href="https://#Exercise E - Spectrogram">Exercise E - Spectrogram</a></li>
        </ol>
    </li>
    <li><a href="https://#Example 2 - Gene Family Classification">Example 2 - Gene Family Classification</a>
        <ol>
            <li><a href="https://#Exercise A - Load human DNA data">Exercise A - Load human DNA data</a></li>
            <li><a href="https://#Exercise B - Sequences of Characters into K-mers">Exercise B - Sequences of Characters into K-mers</a></li>
            <li><a href="https://#Exercise C - Bag-of-Words">Exercise C - Bag-of-Words</a></li>
            <li><a href="https://#Exercise D - Multinomial Naïve Bayes Classifier">Exercise D - Multinomial Naïve Bayes Classifier</a></li>
        </ol>
    </li>
</ol>


***


## Setup


For this project, the following libraries were used:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (like Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the following code cell.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

The following required libraries are **not** pre-installed in the Skills Network Labs environment. **You will need to run the following cell** to install them:


In [ ]:
%%capture

!pip install nltk 

### Importing Required Libraries


In [ ]:
import warnings
warnings.simplefilter('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
print(tf. __version__)
import skillsnetwork

import keras 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import os
import pathlib
from scipy import signal
from scipy.io import wavfile
import re
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

sns.set_context('notebook')
sns.set_style('white')

## Background

In machine learning, it is quite common to assume that the data are identically and independently distributed or i.i.d. This implies that the generative process does not have any memory of past samples to generate new samples.
This assumption is usually violated when dealing with sequential data as samples depend on past information, that is, they are reliant on one another due to their sequential order.
Examples where data depends on past information may include weather. For instance, it's not likely to snow if the temperature is extremely hot the day before. In financial time series, the price of the stock is not only determined by the fundamentals of the company but the price of the stock the day before; we will see more examples later on.
There are many method to study time series data; in this section, we will study some of the classic methods. These methods are precursors to Recurrent Neural Networks (RNNs), a well-known method, well-suited to model these forms of sequential data that we will cover in the next few sections.


## The i.i.d. Assumption


This project, is dealing with data that, rather than being drawn i.i.d., from some joint distribution $P(x, y)$, actually consists of sequences of $(x, y)$ pairs that show some sequential correlation. This means that values close to the x and y values are likely to be related to and/or dependent on each other.


So when the points in the data set are dependent on the other points in the data set, the data is termed sequential.


The sequential supervised learning problem can be formulated as follows:


Let ${(x_i, y_i)}^N\_{i=1}$ be a set of N training examples. In a part-of-speech tagging task, one $(x_i, y_i)$ pair might consist of $x_i =$ do you want fries and $y_i =$ verb pronoun verb noun. Our goal is to build a model, $h$, that predicts the next label sequence, $y = h(x)$, given an input sequence, $x$. Here the task is to predict the $t+1$st element of the sequence $(y\_1,...,y_t)$.


Let us walk through an example to highlight the issues when working with non-i.i.d. data. We will be using financial quotations from various energy companies.


In [ ]:
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/datasets/financial-data.zip")


In [ ]:
import pandas as pd

symbols = {"TOT": "Total", "XOM": "Exxon", "CVX": "Chevron",
           "COP": "ConocoPhillips", "VLO": "Valero Energy"}
template_name = ("./financial-data/{}.csv")

quotes = {}
for symbol in symbols:
    data = pd.read_csv(
        template_name.format(symbol), index_col=0, parse_dates=True
    )
    quotes[symbols[symbol]] = data["open"]
quotes = pd.DataFrame(quotes)
quotes .head()

Let us plot a few financial quotations.


In [ ]:
import matplotlib.pyplot as plt

quotes.plot()
plt.ylabel("Quote value")
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = plt.title("Stock values over time")

We want to predict the quotation of Chevron using all other energy companies’ quotes. We will use the decision tree regressor that is expected to overfit and thus not generalize to unseen data. We will start off by splitting our data into a testing and training set.


In [ ]:
data, target = quotes.drop(columns=["Chevron"]), quotes["Chevron"]
data_train, data_test, target_train, target_test = train_test_split(
    data, target, shuffle=True, random_state=0)

we have our features


In [ ]:
 data

and target Chevron


In [ ]:
target

Let’s now define our model:


In [ ]:
regressor = DecisionTreeRegressor()

And now we apply a cross-validation strategy, to check the generalization performance of our model.


In [ ]:
cv = ShuffleSplit(random_state=0)

In [ ]:
regressor.fit(data_train, target_train)
target_predicted = regressor.predict(data_test)
# Affect the index of `target_predicted` to ease the plotting
target_predicted = pd.Series(target_predicted, index=target_test.index)
target_predicted 

Now we perform evaluation.


In [ ]:
from sklearn.metrics import r2_score

test_score = r2_score(target_test, target_predicted)
print(f"The R2 on this single split is: {test_score:.2f}")

We get outstanding generalization performance in terms of $R^2$. Let's plot the predictions against the ground truth.


In [ ]:
target_train.plot(label="Training")
target_test.plot(label="Testing")
target_predicted.plot(label="Prediction")

plt.ylabel("Quote value")
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = plt.title("Model predictions using a ShuffleSplit strategy")

The results are surprisingly very good, as we had originally expected the model to overfit and not generalize on unseen test sets. Let us now investigate why this is the case.

We used a cross-validation method that shuffles data and splits. We will simplify this procedure by not shuffling our data and plotting the results obtained.


In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, shuffle=False, random_state=0,
)
regressor.fit(data_train, target_train)
target_predicted = regressor.predict(data_test)
target_predicted = pd.Series(target_predicted, index=target_test.index)

In [ ]:
test_score = r2_score(target_test, target_predicted)
print(f"The R2 on this single split is: {test_score:.2f}")

In [ ]:
target_train.plot(label="Training")
target_test.plot(label="Testing")
target_predicted.plot(label="Prediction")

plt.ylabel("Quote value")
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = plt.title("Model predictions using a split without shuffling")

Now we see that our model performs worse than just predicting the mean of the target. Why is that?


In this dataset, and all other similar time series datasets, the testing samples follow some of the training sample. As mentioned before, in time-series subsequent samples are dependent on previous samples; that is, there is a relationship between sample $t$ and sample $t+1$. We are clearly violating the i.i.d. assumption. The model's prediction would be close to true values even when it doesn't learn anything, because of data leakage and because it could have just memorized the training dataset.


## Understanding Sequential Modeling


Sequential modeling can be described as the process of forecasting a sequence of values from a set of input values, similar to what we saw in the previous section.

For example, consider a time-series that represents how a patient's physiological values like continuous heart rate, ECG, breathing rate, and temperature, change over time. One possible sequential modeling task could be to forecast their disease trajectory or predict their medical condition at some future time.


Text prediction is another example. For example, you can be given a sequence of words, and your modeling task could be to predict the next word based on the sequence of the previous word/phrase and a set of pre-loaded conditions and rules.


## Forms of Sequential Data

Sequential data contains elements that are ordered into sequences. For example, time series (like stock values or sensor measurements), gene sequences (${C,G,A,T}$), speech, text (${a,...,z,0,...,9,...}$), video clips, and musical notes, and so on.

To summarize, sequential data has some temporal coherence, and can be of arbitrary lengths. A lot of tasks can be modeled from these types of data. For example:

*   text classification, such as spam email or not
*   language translation, such as French to English
*   time-series forecasting, such as stock prices prediction

Let us look at a few common sequential data sets, and understand pre-processing techniques associated with each.


### Working with time-series data


Time series are special types of sequences that consist of random variables indexed by time. In particular, the random variables can be dependent and their distribution might change over time, so time-series also violate the i.i.d. assumption.


#### Time-series decomposition


Time series data can exhibit various patterns, and it is often helpful to split a time series into several components, each representing an underlying pattern category. These include trend, seasonality, and cycles, which we will now explain using examples.


*   **trend**: A trend is observed when there is an increasing or decreasing slope observed in the time series. In the following time series, we see an increasing pattern.


In [ ]:
trend = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/guinearice.csv', parse_dates=['date'], index_col='date')
plt.plot(trend.values)

*   **seasonality**: Seasonality represents a distinct repeated pattern observed between regular intervals due to seasonal factors, like month of the year, the day of the month, weekdays or even time of the day, festivals, and so on.


In [ ]:
seasonality = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/sunspotarea.csv', parse_dates=['date'], index_col='date')
plt.plot(seasonality.value)

A time series can have both trend and seasonality, for example:


In [ ]:
seasonality_trend = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/AirPassengers.csv', parse_dates=['date'], index_col='date')
plt.plot(seasonality_trend.value)

*   **cyclic**: Cyclicity is similar to seasonality, but it happens when the rise and fall pattern in the series does not happen in fixed calendar-based intervals, that is, if the patterns are not of fixed calendar-based frequencies, then the pattern is cyclic.


A time series can be modeled as an additive or multiplicative, wherein each observation in the series can be expressed as either a sum or a product of the components.

**Additive time series** \
\
$Value(t) = BaseLevel(t) + Trend(t) + Seasonality(t) + Error(t)$

**Multiplicative Time Series** \
\
$Value(t) = BaseLevel (t)x Trend(t) x Seasonality(t) x Error(t)$


We can use functions like `seasonal_decompose` from the python package, `statsmodel`, to decompose our time-series into trend, seasonality, and the residual components.


In [ ]:
# Import Data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/a10.csv', parse_dates=['date'], index_col='date')
plt.plot(df.value)

Let us start by performing additive decomposition. Note that by using `extrapolate_trend = 'freq'`, we impute missing values at the beginning of the time series.


In [ ]:
# Additive Decomposition
result_add = seasonal_decompose(df['value'], model='additive', extrapolate_trend='freq')

# Plot
result_add.plot()

Now, similarly we can try performing multiplicative decomposition:


In [ ]:
# Multiplicative Decomposition 
result_mul = seasonal_decompose(df['value'], model='multiplicative', extrapolate_trend='freq')

#Plot
result_mul.plot()

In this particular case, we see that with additive decomposition, some pattern is still left over, while with multiplicative decomposition, the result looks quite random, which is what we want.


So how do we detrend our time series? One way would be to subtract the trend component obtained from the time series decomposition we saw earlier.


In [ ]:
detrended = df.value.values - result_mul.trend
plt.plot(detrended)
plt.title('Drug Sales detrended by subtracting the trend component', fontsize=16)

Similarly, we can remove the seasonality component:


In [ ]:
deseasonalized = df.value.values / result_mul.seasonal

# Plot
plt.plot(deseasonalized)

#### Time-series imputation


Earlier, we briefly mentioned imputing missing values in a time-series. Sometimes, your time series will have missing dates/times. That means the data was not captured or unavailable for those periods.

Depending on the nature of the series, we can try multiple approaches for imputation:

*   Forward
*   Backward Fill
*   Linear Interpolation, etc.


Let us start by parsing the data frame indices as dates.


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/a10.csv', parse_dates=['date'], index_col='date')

*   **Forward Imputation**


In [ ]:
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error

df_ffill = df.ffill()
# Print the MSE between imputed value and ground truth
error = np.round(mean_squared_error(df['value'], df_ffill['value']), 2)
df_ffill['value'].plot(title='Forward Fill (MSE: ' + str(error) +")", label='Forward Fill', style=".-")

*   **Backward Imputation**


In [ ]:
df_bfill = df.bfill()
error = np.round(mean_squared_error(df['value'], df_bfill['value']), 2)
df_bfill['value'].plot(title="Backward Fill (MSE: " + str(error) +")", label='Back Fill', color='firebrick', style=".-")


*   **Linear Interpolation**


In [ ]:
df['rownum'] = np.arange(df.shape[0])
df_nona = df.dropna(subset = ['value'])
f = interp1d(df_nona['rownum'], df_nona['value'])
df['linear_fill'] = f(df['rownum'])
error = np.round(mean_squared_error(df['value'], df['linear_fill']), 2)
df['linear_fill'].plot(title="Linear Fill (MSE: " + str(error) +")",label='Cubic Fill', color='green', style=".-")


Time-series decomposition and imputation are common pre-processing steps used when working on time-series prediction or forecasting tasks.


### Working with text data


In this section, we will be working with a portion of the [Twitter Sentiment Analysis](https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01) dataset from Kaggle.

> Given a message and an entity, the task is to judge the sentiment of the message about the entity. There are three classes in this dataset: Positive, Negative, and Neutral.

The file contains the tweet along with the labels. For the purpose of this lab, we will just explore the dataset, and perform some basic pre-processing.


First, let us download this dataset.


In [ ]:
cols = ['Id', 'Entity', 'Sentiment', 'Tweet']
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/twitter_validation.csv', names = cols, header=None)

In [ ]:
df.head(2)

#### Visualizing


Using `groupby` in pandas, we can determine the number of tweets present in each category and plot them.


In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby(['Sentiment']).Tweet.count().sort_values().plot.barh(
    ylim=0, title= 'tweets per category')

plt.xlabel('# of occurrences', fontsize = 12);

#### Cleaning up


Now we will focus on preprocessing the raw tweets. This step is important because raw tweets without preprocessing are highly unstructured and contain redundant and often problematic information. They contain tons of noise that should be removed.


Most of the text data is cleaned by following the steps below.

*   Remove punctuations
*   Remove stopwords
*   Tokenization - Converting a sentence into list of words
*   Lemmatization and stemming - Tranforming any form of a word to its root word


Using a corpus of stop-words, including articles, prepositions, and conjunctions, from the Natural Language Toolkit (NLTK) suite, words with little lexical content (such as the, a, also, from, and so on) can be filtered out, for more check out this <a href="https://realpython.com/nltk-nlp-python/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01">link</a>


In [ ]:
def remove_punctuation(text):
    regular_punct = list(string.punctuation)
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip().lower()

df['Tweet'] = df['Tweet'].apply(remove_punctuation)

In [ ]:
def remove_stopwords(tweet):
    en_stops = set(stopwords.words('english'))
    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in en_stops])  
    return tweet

df['Tweet'] = df['Tweet'].apply(remove_stopwords)

Tweets can be further processed with tokenization and lemmatization, both of which were performed using packages in the NLTK suite.

Tokenization converts text to analysis relevant word tokens, while lemmatization transforms words to a simpler form, returning the word’s lemma – a canonical form of all its inflectional forms (for example, go represents its inflected forms of goes, going, went, gone).


In [ ]:
df['Tweet'] = df['Tweet'].apply(word_tokenize)

In [ ]:
def lemma_wordnet(input):
    lem = WordNetLemmatizer()
    return [lem.lemmatize(w) for w in input]

df['Tweet'] = df['Tweet'].apply(lemma_wordnet)

#### Vectorization


Here we explore two techniques for encoding text data into numerical vectors. This is to extract suitable features from processed texts. Both will be implemented using scikitlearn’s CountVectorizer, where `ngram_range` will be set to (1,1) for BoW and (2,2) for B-BoW.


Bag-of-Words (BoW) is a vectorization method to convert variable-length texts into fixed-length vectors, without considering the semantic relation between words.


BoW often results in a very sparse representation. For a large dataset with the vocabulary of a few thousand words, preprocessing text before employing BoW can be useful.


In [ ]:
# combine individual words

def combine_text(input):
    combined = ' '.join(input)
    return combined

df['Tweet'] = df['Tweet'].apply(combine_text)

We can plot some of the values we see they are zero


In [ ]:
cv = CountVectorizer(ngram_range=(1, 1))
X_train_bow = cv.fit_transform(df['Tweet'])
print(X_train_bow[0:10,0:10].todense())

In [ ]:
cv = CountVectorizer(ngram_range=(1, 1))
X_train_bow = cv.fit_transform(df['Tweet'].values.tolist()) 
Y_train_bow = df['Sentiment']
Y_train_bow

Bigram BoW (B-BoW) represents a text document as a weakly ordered collection of contiguous sequences but of two items. It allows for the preservation of more word locality information.


In [ ]:
cv_bbow = CountVectorizer(ngram_range=(2, 2))
X_train_bbow = cv_bbow.fit_transform(df['Tweet']) 
Y_train_bbow = df['Sentiment']
Y_train_bbow

Term frequency-inverse document frequency (TF-IDF) is a frequency-based embedding method that
measures the importance of a word in a given document. It can be implemented using the `TfidfVectorizer` function from sklearn.


In [ ]:
vectorizer = TfidfVectorizer(use_idf = True, ngram_range=(1, 1))
vectorizer.fit(df)
X_train_tfidf = vectorizer.fit_transform(df['Tweet'])
Y_train_tfidf = df['Sentiment'] 
Y_train_tfidf 